In [6]:
using LinearAlgebra
using Distributions
using Plots

function quadrotor_dynamics(state, inputs, dt)
    # Unpack the state
    x, y, z, vx, vy, vz, phi, theta, psi, p, q, r = state
    
    # Unpack the inputs
    thrust, tau_phi, tau_theta, tau_psi = inputs
    
    # Constants
    m = 1.0  # Mass of the quadrotor
    g = 9.81  # Gravitational acceleration
    Ix = 0.01  # Moment of inertia around x-axis
    Iy = 0.01  # Moment of inertia around y-axis
    Iz = 0.02  # Moment of inertia around z-axis
    
    # Translational dynamics
    ax = 0
    ay = 0
    az = thrust / m - g
    
    # Rotational dynamics
    p_dot = tau_phi / Ix
    q_dot = tau_theta / Iy
    r_dot = tau_psi / Iz
    
    # Integrate to get the new state
    new_state = [
        x + vx * dt,
        y + vy * dt,
        z + vz * dt,
        vx + ax * dt,
        vy + ay * dt,
        vz + az * dt,
        phi + p * dt,
        theta + q * dt,
        psi + r * dt,
        p + p_dot * dt,
        q + q_dot * dt,
        r + r_dot * dt
    ]
    
    return new_state
end

function linearize(state_eq, input_eq, dt)
    n_states = length(state_eq)
    n_inputs = length(input_eq)
    
    A = zeros(n_states, n_states)
    B = zeros(n_states, n_inputs)
    
    epsilon = 1e-6
    
    # Compute A matrix
    for i in 1:n_states
        delta = zeros(n_states)
        delta[i] = epsilon
        state_plus = state_eq + delta
        state_minus = state_eq - delta
        
        f_plus = quadrotor_dynamics(state_plus, input_eq, dt)
        f_minus = quadrotor_dynamics(state_minus, input_eq, dt)
        
        A[:, i] = (f_plus - f_minus) / (2 * epsilon)
    end
    
    # Compute B matrix
    for i in 1:n_inputs
        delta = zeros(n_inputs)
        delta[i] = epsilon
        input_plus = input_eq + delta
        input_minus = input_eq - delta
        
        f_plus = quadrotor_dynamics(state_eq, input_plus, dt)
        f_minus = quadrotor_dynamics(state_eq, input_minus, dt)
        
        B[:, i] = (f_plus - f_minus) / (2 * epsilon)
    end
    
    return A, B
end

function kalman_filter(A, B, B_d, Q, R, x_hat, P, u, y, wind_mean, wind_covariance)
    d_t = rand(MvNormal(wind_mean, Symmetric(wind_covariance)))
    
    # Prediction Step
    x_hat_minus = A * x_hat + B * u + B_d * d_t
    P_minus = A * P * transpose(A) + Q
    
    # Update Step
    C = I(size(x_hat, 1))  # Assuming we are measuring all states
    K = P_minus * transpose(C) * inv(C * P_minus * transpose(C) + R)  # Kalman gain
    x_hat_plus = x_hat_minus + K * (y - C * x_hat_minus)
    P_plus = (I(size(P, 1)) - K * C) * P_minus
    
    return x_hat_plus, P_plus
end

kalman_filter (generic function with 1 method)

In [ ]:
# Simulation parameters
dt = 0.01  # Time step
total_time = 10  # Total simulation time
time = 0:dt:total_time  # Simulation time array
n_states = 12  # Number of states
n_inputs = 4  # Number of inputs

# Equilibrium state and input (hovering)
state_eq = zeros(n_states)
input_eq = [9.81, 0, 0, 0]  # Input to counteract gravity

# Disturbance (wind) model parameters
wind_mean = zeros(n_states)  # Assume wind has zero mean
wind_covariance = diagm(0 => [0.01 for _ in 1:3] .+ [0.0 for _ in 4:12])  # Variance in the translational states (x, y, z)

# System matrices (would be computed from the system's dynamics)
A, B = linearize(state_eq, input_eq, dt)
B_d = 100 * ones(n_states, n_states)  # Effect of disturbance on the state
# B_d[1:3, 1:3] = I(3)  # Assuming wind affects only the translational states

# Noise covariances
Q = I(n_states) * 0.01  # Process noise covariance
R = I(n_states) * 0.1  # Measurement noise covariance

# Initial state and covariance
x_hat = zeros(n_states)  # Initial estimate
P = I(n_states)  # Initial estimate covariance

# Storage for simulation results
true_states = []
estimated_states = []

for t in time
    # Simulate true dynamics with random wind disturbance
    wind_disturbance = rand(MvNormal(wind_mean, Symmetric(wind_covariance)))
    true_state = quadrotor_dynamics(x_hat, input_eq, dt) + B_d * wind_disturbance
    push!(true_states, true_state)
    
    # Generate noisy measurements
    measurements = true_state + randn(n_states) * 0.1
    
    # Kalman filter to estimate the state
    x_hat, P = kalman_filter(A, B, B_d, Q, R, x_hat, P, input_eq, measurements, wind_mean, wind_covariance)
    push!(estimated_states, x_hat)
end

# Plotting
println("*** Plotting States ***")
p = plot(layout = (4, 3), size = (1200, 800))
labels = ["x", "y", "z", "vx", "vy", "vz", "phi (Roll)", "theta (Pitch)", "psi (Yaw)", "p (Angular Vel X)", "q (Angular Vel Y)", "r (Angular Vel Z)"]

for i in 1:n_states
    plot!(p[i], time, getindex.(true_states, i), label="True")
    plot!(p[i], time, getindex.(estimated_states, i), label="Estimated", linestyle=:dash)
    xlabel!(p[i], "Time (s)")
    ylabel!(p[i], labels[i])
    legend!(p[i])
end

display(p)